In [4]:
library(lme4)
library(lmerTest)
library(emmeans)
library(repr)
library(glue)
library(performance)
library(IRdisplay)
library(comprehenr)
library(GLMMadaptive)
library(ordinal)
library(multilevelTools)

In [5]:
options(width = 160)

In [6]:
current_participants_df <- read.csv('./human_evals_data/current_participants_df.csv')

# Set columns as categorical
current_participants_df$participant_id <- as.factor(current_participants_df$participant_id)
current_participants_df$game_id <- as.factor(current_participants_df$game_id)
current_participants_df$game_type <- as.factor(current_participants_df$game_type)
current_participants_df$full_game_id <- as.factor(current_participants_df$full_game_id)

# Change the order of game_type levels
current_participants_df$game_type <- factor(current_participants_df$game_type, levels = c('unmatched', 'unmatched_top_30', 'matched', 'real'))

# Set columns as booleans
current_participants_df$real <- as.logical(current_participants_df$real)
current_participants_df$matched <- as.logical(current_participants_df$matched)

ATTRIBUTES <- c("confident", "fun_play", "fun_watch", "capability", "goldilocks", "creativity", "human_likeness")
NORMALIZED_ATTRIBUTES <- to_vec(for (attr in ATTRIBUTES) { glue("normalized_{attr}") } )
CENTERED_ATTRIBUTES <- to_vec(for (attr in ATTRIBUTES) { glue("centered_{attr}") } )
FULL_NORMALIZED_ATTRIBUTES <- to_vec(for (attr in ATTRIBUTES) { glue("full_normalized_{attr}") })

for (i in seq_along(CENTERED_ATTRIBUTES)) {
    current_participants_df[[CENTERED_ATTRIBUTES[i]]] <- current_participants_df[[ATTRIBUTES[i]]] - 3
}

for (i in seq_along(FULL_NORMALIZED_ATTRIBUTES)) {
    current_participants_df[[FULL_NORMALIZED_ATTRIBUTES[i]]] <- (current_participants_df[[ATTRIBUTES[i]]] - mean(current_participants_df[[ATTRIBUTES[i]]])) / sd(current_participants_df[[ATTRIBUTES[i]]])
}



print(dim(current_participants_df))
head(current_participants_df, 2)

[1] 1292   45


,participant_id,game_id,real,matched,confident,fun_play,fun_watch,capability,goldilocks,creativity,...,centered_goldilocks,centered_creativity,centered_human_likeness,full_normalized_confident,full_normalized_fun_play,full_normalized_fun_watch,full_normalized_capability,full_normalized_goldilocks,full_normalized_creativity,full_normalized_human_likeness
,<fct>,<fct>,<lgl>,<lgl>,<int>,<int>,<int>,<int>,<int>,<int>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,00c9bf44-28f3-469a-8a71-ea972af61bab-p102,"(1, 1, 4, 0, 2, 0, 0, 0, 1, 0, 0, 0)",FALSE,FALSE,4,3,3,5,3,2,...,0,-1,-1,0.3349299,0.685856,0.736462,1.7028050,0.3154651,-0.1941839,-0.3231405
2,00c9bf44-28f3-469a-8a71-ea972af61bab-p102,"(1, 0, 2, 0, 1, 0, 0, 0, 0, 1, 0, 0)",FALSE,FALSE,4,3,3,4,3,3,...,0,0,-1,0.3349299,0.685856,0.736462,0.8894874,0.3154651,0.7489952,-0.3231405


In [7]:
filtered_df <- current_participants_df[current_participants_df$game_type != 'unmatched', ]
filtered_df$game_type <- droplevels(filtered_df$game_type)

print(dim(filtered_df))
head(filtered_df, 2)

[1] 892  45


,participant_id,game_id,real,matched,confident,fun_play,fun_watch,capability,goldilocks,creativity,...,centered_goldilocks,centered_creativity,centered_human_likeness,full_normalized_confident,full_normalized_fun_play,full_normalized_fun_watch,full_normalized_capability,full_normalized_goldilocks,full_normalized_creativity,full_normalized_human_likeness
,<fct>,<fct>,<lgl>,<lgl>,<int>,<int>,<int>,<int>,<int>,<int>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
3,00c9bf44-28f3-469a-8a71-ea972af61bab-p102,"(1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0)",TRUE,TRUE,4,2,2,5,4,3,...,1,0,-1,0.3349299,-0.2092207,-0.1445648,1.70280498,1.2524326,0.7489952,-0.3231405
4,00c9bf44-28f3-469a-8a71-ea972af61bab-p102,"(1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0)",FALSE,TRUE,5,4,4,3,2,2,...,-1,-1,-1,1.1049113,1.5809328,1.6174888,0.07616983,-0.6215025,-0.1941839,-0.3231405


## Do we need a mixed effects model?

In [8]:
# Iterate through attributes
differences <- c()
for (attr in FULL_NORMALIZED_ATTRIBUTES) {
    # Create formula with current attribute
    attr.glm <- glm(formula = as.formula(glue("{attr} ~ 1 + game_type + normalized_fitness")), data = filtered_df)
    attr.lmer <- lmer(formula = as.formula(glue("{attr} ~ 1 + game_type + normalized_fitness + (1 | participant_id)")), data = filtered_df)
    
    fixed_effects_aic <- AIC(logLik(attr.glm))
    mixed_effects_aic <- AIC(logLik(attr.lmer))
    aic_diff <- fixed_effects_aic - mixed_effects_aic
    differences <- append(differences, aic_diff)
    print(glue("AIC for {attr} Fixed effects model: {format(fixed_effects_aic, nsmall=3)} | ME with game id: {format(mixed_effects_aic, nsmall=3)} | Difference: {format(aic_diff, nsmall=3)}"))
}

mean_diff <- mean(differences)
print(glue("Mean difference: {format(mean_diff, nsmall=3)}"))

# anova(conf.model.without, conf.model.with)
# summary(confident_model)

AIC for full_normalized_confident Fixed effects model: 2365.468 | ME with game id: 2310.450 | Difference: 55.0183
AIC for full_normalized_fun_play Fixed effects model: 2509.595 | ME with game id: 2284.551 | Difference: 225.0439
AIC for full_normalized_fun_watch Fixed effects model: 2541.932 | ME with game id: 2164.678 | Difference: 377.2531
AIC for full_normalized_capability Fixed effects model: 2476.032 | ME with game id: 2162.976 | Difference: 313.0557
AIC for full_normalized_goldilocks Fixed effects model: 2416.218 | ME with game id: 2376.805 | Difference: 39.41309
AIC for full_normalized_creativity Fixed effects model: 2483.085 | ME with game id: 2266.051 | Difference: 217.0344
AIC for full_normalized_human_likeness Fixed effects model: 2463.446 | ME with game id: 2310.330 | Difference: 153.1154
Mean difference: 197.1334


Conclusion: if we don't normalize scores by participant, we absolutely need a per-participant random effect

## Do we need a mixed effect for the games?

In [9]:
# Iterate through attributes
for (attr in FULL_NORMALIZED_ATTRIBUTES) {
    # Create formula with current attribute
    attr.lmer_no_game_id <- lmer(formula = as.formula(glue("{attr} ~ 1 + game_type + normalized_fitness + (1 | participant_id)")), data = filtered_df)
    attr.lmer_with_game_id <- update(attr.lmer_no_game_id, . ~ . + (1 + game_type | full_game_id))
    
    anova_results <- anova(attr.lmer_no_game_id, attr.lmer_with_game_id)
    attr.aic_diff <- anova_results$AIC[[1]] - anova_results$AIC[[2]]
    attr.chisq <- anova_results$Chisq[[2]]
    attr.pvalue <- anova_results$`Pr(>Chisq)`[[2]] 
    # fixed_effects_aic <- AIC(logLik(attr.glm))
    # mixed_effects_aic <- AIC(logLik(attr.lmer))
    # aic_diff <- fixed_effects_aic - mixed_effects_aic
    print(glue("For {attr}, adding the game ID random effects results in AIC diff: {format(attr.aic_diff, nsmall=3)} | Chi-squared: {format(attr.chisq, nsmall=3)} | p-value: {format(attr.pvalue, nsmall=3)}"))

}

# anova(conf.model.without, conf.model.with)
# summary(confident_model)

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"unable to evaluate scaled gradient"
Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model failed to converge: degenerate  Hessian with 1 negative eigenvalues"
Warning message:
"Model failed to converge with 1 negative eigenvalue: -6.9e-03"
refitting model(s) with ML (instead of REML)



For full_normalized_confident, adding the game ID random effects results in AIC diff: 54.84572 | Chi-squared: 66.84572 | p-value: 1.809949e-12


Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"unable to evaluate scaled gradient"
Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model failed to converge: degenerate  Hessian with 3 negative eigenvalues"
Warning message:
"Model failed to converge with 3 negative eigenvalues: -1.9e-03 -8.2e-03 -8.5e-03"
refitting model(s) with ML (instead of REML)



For full_normalized_fun_play, adding the game ID random effects results in AIC diff: 31.06281 | Chi-squared: 43.06281 | p-value: 1.133526e-07


Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"unable to evaluate scaled gradient"
Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model failed to converge: degenerate  Hessian with 1 negative eigenvalues"
Warning message:
"Model failed to converge with 1 negative eigenvalue: -5.4e-04"
refitting model(s) with ML (instead of REML)



...

For full_normalized_goldilocks, adding the game ID random effects results in AIC diff: 97.39101 | Chi-squared: 109.391 | p-value: 2.733972e-21


refitting model(s) with ML (instead of REML)



For full_normalized_creativity, adding the game ID random effects results in AIC diff: 46.15622 | Chi-squared: 58.15622 | p-value: 1.065341e-10


Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"unable to evaluate scaled gradient"
Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model failed to converge: degenerate  Hessian with 1 negative eigenvalues"
Warning message:
"Model failed to converge with 1 negative eigenvalue: -3.7e-03"
refitting model(s) with ML (instead of REML)



For full_normalized_human_likeness, adding the game ID random effects results in AIC diff: 22.09407 | Chi-squared: 34.09407 | p-value: 6.451789e-06


Conclusion: we absolutely also need a mixed effect for the participants

## Let's analyze these now

In [10]:
for (attr in FULL_NORMALIZED_ATTRIBUTES) {
    display_markdown(glue("## {attr}"))

    # Create formula with current attribute
    attr.lmer <- lmer(formula = as.formula(glue("{attr} ~ 1 + game_type + normalized_fitness + (1 | participant_id) + (game_type | full_game_id)")), data = filtered_df)

    display_markdown("### Mixed Effects Model")    
    display(summary(attr.lmer))

    attr.marginal_means = emmeans(attr.lmer, "game_type")
    display_markdown("### Marginal Means")
    display(pairs(attr.marginal_means, reverse = TRUE))

    display_markdown("------------------")

    # anova_results <- anova(attr.lmer_no_participant, attr.lmer_with_participant)
    # attr.chisq <- anova_results$Chisq[[2]]
    # attr.pvalue <- anova_results$`Pr(>Chisq)`[[2]] 
    # # fixed_effects_aic <- AIC(logLik(attr.glm))
    # # mixed_effects_aic <- AIC(logLik(attr.lmer))
    # # aic_diff <- fixed_effects_aic - mixed_effects_aic
    # print(glue("For {attr}, adding the participant ID random effects results in Chi-squared: {format(attr.chisq, nsmall=3)} | p-value: {format(attr.pvalue, nsmall=3)}"))

}


## full_normalized_confident

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"unable to evaluate scaled gradient"
Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model failed to converge: degenerate  Hessian with 1 negative eigenvalues"
Warning message:
"Model failed to converge with 1 negative eigenvalue: -6.9e-03"


### Mixed Effects Model

Linear mixed model fit by REML. t-tests use Satterthwaite's method ['lmerModLmerTest']
Formula: as.formula(glue("{attr} ~ 1 + game_type + normalized_fitness + (1 | participant_id) + (game_type | full_game_id)"))
   Data: filtered_df

REML criterion at convergence: 2228.3

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.7516 -0.5731  0.1287  0.6818  2.0203 

Random effects:
 Groups         Name             Variance Std.Dev. Corr       
 participant_id (Intercept)      0.15285  0.3910              
 full_game_id   (Intercept)      0.05024  0.2241              
                game_typematched 0.32362  0.5689   -0.87      
                game_typereal    0.33333  0.5773   -0.67  0.56
 Residual                        0.54876  0.7408              
Number of obs: 892, groups:  participant_id, 100; full_game_id, 90

Fixed effects:
                   Estimate Std. Error       df t value Pr(>|t|)    
(Intercept)        -0.17962    0.07261 44.37840  -2.474  0.01726 *  
game_typem

### Marginal Means

...

### Mixed Effects Model

Linear mixed model fit by REML. t-tests use Satterthwaite's method ['lmerModLmerTest']
Formula: as.formula(glue("{attr} ~ 1 + game_type + normalized_fitness + (1 | participant_id) + (game_type | full_game_id)"))
   Data: filtered_df

REML criterion at convergence: 2261.9

Scaled residuals: 
     Min       1Q   Median       3Q      Max 
-2.60218 -0.66011 -0.05462  0.61999  2.90758 

Random effects:
 Groups         Name             Variance Std.Dev. Corr       
 participant_id (Intercept)      0.29712  0.5451              
 full_game_id   (Intercept)      0.02357  0.1535              
                game_typematched 0.09642  0.3105   -0.31      
                game_typereal    0.22700  0.4764   -0.79  0.22
 Residual                        0.55569  0.7454              
Number of obs: 892, groups:  participant_id, 100; full_game_id, 90

Fixed effects:
                   Estimate Std. Error       df t value Pr(>|t|)    
(Intercept)        -0.20740    0.07612 71.33836  -2.725  0.00809 ** 


### Marginal Means

 contrast                   estimate     SE   df t.ratio p.value
 matched - unmatched_top_30    0.301 0.0945 51.5   3.181  0.0069
 real - unmatched_top_30       0.559 0.0971 44.8   5.751  <.0001
 real - matched                0.258 0.1135 56.0   2.272  0.0682

Degrees-of-freedom method: kenward-roger 
P value adjustment: tukey method for comparing a family of 3 estimates 

------------------

In [17]:
strictControl <- lmerControl(optCtrl = list(
   algorithm = "NLOPT_LN_NELDERMEAD",
   xtol_abs = 1e-12,
   ftol_abs = 1e-12))

l <- lmer(formula = full_normalized_confident ~ 1 + normalized_fitness + (1 | participant_id) + (1 | game_type/full_game_id), 
    data = filtered_df, control = strictControl)

summary(l)
md <- modelDiagnostics(l, ev.perv = 0.001)
plot(md, ask = FALSE)

Linear mixed model fit by REML. t-tests use Satterthwaite's method ['lmerModLmerTest']
Formula: full_normalized_confident ~ 1 + normalized_fitness + (1 | participant_id) +      (1 | game_type/full_game_id)
   Data: filtered_df
Control: strictControl

REML criterion at convergence: 2234.3

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.7622 -0.5870  0.1217  0.6856  1.9656 

Random effects:
 Groups                 Name        Variance Std.Dev.
 participant_id         (Intercept) 0.15154  0.3893  
 full_game_id:game_type (Intercept) 0.13691  0.3700  
 game_type              (Intercept) 0.04603  0.2145  
 Residual                           0.54934  0.7412  
Number of obs: 892, groups:  participant_id, 100; full_game_id:game_type, 90; game_type, 3

Fixed effects:
                   Estimate Std. Error       df t value Pr(>|t|)    
(Intercept)         0.04398    0.13854  2.38694   0.317    0.777    
normalized_fitness  0.31766    0.05771 86.07059   5.504 3.77e-07 ***
---
Sign

ERROR: Error in modelDiagnostics(l, ev.perv = 0.001): could not find function "modelDiagnostics"


In [18]:
l

Linear mixed model fit by REML ['lmerModLmerTest']
Formula: full_normalized_confident ~ 1 + normalized_fitness + (1 | participant_id) +      (1 | game_type/full_game_id)
   Data: filtered_df
REML criterion at convergence: 2234.303
Random effects:
 Groups                 Name        Std.Dev.
 participant_id         (Intercept) 0.3893  
 full_game_id:game_type (Intercept) 0.3700  
 game_type              (Intercept) 0.2145  
 Residual                           0.7412  
Number of obs: 892, groups:  participant_id, 100; full_game_id:game_type, 90; game_type, 3
Fixed Effects:
       (Intercept)  normalized_fitness  
           0.04398             0.31766  

In [ ]:
confident_means = emmeans(confident_model, "game_type")
print(pairs(confident_means, reverse = TRUE))
print(pwpm(confident_means, reverse = TRUE))

In [ ]:
mm <- clmm(C(confident) ~ 1+ C(game_type) + normalized_fitness + (1 | participant_id),
    # random = d +  , # + (1 | full_game_id), 
    data = filtered_df,
    # Hess = TRUE,
    )

summary(mm)

In [ ]:
mm <- clmm(C(confident) ~ 1+ C(game_type) + normalized_fitness + (1 | participant_id) + (1 | full_game_id),
    # random = d +  , # + (1 | full_game_id), 
    data = filtered_df,
    # Hess = TRUE,
    )

summary(mm)

In [ ]:
install.packages("multilevelTools")